In [2]:
import tensorflow as tf
from pathlib import Path

url = "https://github.com/stevelukis/rnn-ind-eng/raw/main/ind-eng.zip"
path = tf.keras.utils.get_file("ind-eng.zip", origin=url, cache_dir=".",
                               extract=True)
text = (Path(path).parent / "ind.txt").read_text(encoding='utf-8')

In [3]:
import numpy as np

triples = [line.split('\t') for line in text.splitlines()]
np.random.shuffle(triples)
sentences_en, sentences_id, _ = zip(*triples)

In [4]:
print(len(sentences_en))

9243


In [5]:
for i in range(10):
    print(sentences_en[i] + ' => ' + sentences_id[i])

Unbelievable! => Luar biasa!
Tom did it. => Tom melakukannya.
I could be away for some time. => Saya akan pergi selama beberapa waktu.
Tom said that he had to work on Monday. => Tom berkata bahwa dia harus bekerja pada hari Senin.
I thought he was sick. => Kukira dia sakit.
I haven't seen you here before. => Aku belum pernah melihatmu di sini sebelumnya.
It's up to you. => Terserah padamu.
I want to go on a trip with you. => Aku ingin pergi jalan-jalan denganmu.
Who was it that bought this skirt yesterday? => Siapa yang membeli rok ini kemarin?
Tom would never forgive us if we did that. => Tom tidak akan pernah memaafkan kita jika kita melakukan itu.


In [6]:
vocab_size = 500
max_length = 50

text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length
)
text_vec_layer_id = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length
)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_id.adapt([f'startofseq {s} endofseq' for s in sentences_id])
X_train = tf.constant(sentences_en[:8500])
X_val = tf.constant(sentences_en[8500:])
X_train_dec = tf.constant([f'startofseq {s}' for s in sentences_id[:8500]])
X_val_dec = tf.constant([f'startofseq {s}' for s in sentences_id[8500:]])

y_train = text_vec_layer_id([f'{s} endofseq' for s in sentences_id[:8500]])
y_val = text_vec_layer_id([f'{s} endofseq' for s in sentences_id[8500:]])

In [7]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

embed_size = 128

encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_id(decoder_inputs)

encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    embed_size,
                                                    mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [8]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True)
)

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),
                 tf.concat(encoder_state[1::2], axis=-1)]

In [9]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [10]:
output_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
y_proba = output_layer(decoder_outputs)

In [11]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[y_proba])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
              metrics=['accuracy'])
history = model.fit((X_train, X_train_dec), y_train, epochs=20,
                    validation_data=((X_val, X_val_dec), y_val))

Epoch 1/20


2024-01-18 20:23:49.562896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2024-01-18 20:24:03.795412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]


266/266 [==============================] - 437s 2s/step - loss: 3.8938 - accuracy: 0.3247 - val_loss: 3.5071 - val_accuracy: 0.3765
Epoch 2/20
266/266 [==============================] - 378s 1s/step - loss: 3.1637 - accuracy: 0.4119 - val_loss: 3.0718 - val_accuracy: 0.4250
Epoch 3/20
266/266 [==============================] - 362s 1s/step - loss: 2.7911 - accuracy: 0.4543 - val_loss: 2.8375 - val_accuracy: 0.4611
Epoch 4/20
266/266 [==============================] - 404s 2s/step - loss: 2.5213 - accuracy: 0.4866 - val_loss: 2.6642 - val_accuracy: 0.4760
Epoch 5/20
266/266 [==============================] - 412s 2s/step - loss: 2.2801 - accuracy: 0.5166 - val_loss: 2.5268 - val_accuracy: 0.4896
Epoch 6/20
266/266 [==============================] - 431s 2s/step - loss: 2.0549 - accuracy: 0.5433 - val_loss: 2.4182 - val_accuracy: 0.5091
Epoch 7/20
266/266 [==============================] - 368s 1s/step - loss: 1.8458 - accuracy: 0.5749 - val_loss: 2.3477 - val_accuracy: 0.5143
Epoch 8/20

In [ ]:
def translate(sentence_en):
    translation = ''
    for word_idx in range(max_length):
        X = np.array([sentence_en])
        X_dec = np.array([f'startofseq ' + translation])
        y_proba = model.predict([X, X_dec])[0, word_idx]
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_id.get_vocabulary()[predicted_word_id]
        if predicted_word == 'endofseq':
            break
        translation += ' ' + predicted_word
    return translation.strip()

In [ ]:
translate('I go to school today')

1/1 [==============================] - 0s 23ms/step


'aku pergi ke sekolah dengan awal'